# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
with open('data/wiki_data.txt', encoding='utf8')as file:
    corpus = file.read()

In [2]:
from collections import Counter
import re
import textdistance
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [3]:
vocab = Counter(re.findall(r'\w+', corpus.lower()))
from sklearn.feature_extraction.text import CountVectorizer

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [4]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [5]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [ ]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    ##  доработка 1  ##
    if text in vocab:
        return text
    ##  конец доработки 1  ##

    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    ##  доработка 2  ##
    max_dl = max([c[1] for c in closest])

    closest = {
        word: {
            "DL": dist,
            "P" : P(word),
        }
        for word, dist
        in closest
        if dist == max_dl
    }

    return max(closest, key=lambda c: closest[c]["P"])
    ##  конец доработки 2  ##

In [130]:
get_closest_hybrid_match("cсолнце", X, vec, topn=10)

'солнце'

#### Комментарий

Для сравнения возьмем еще Норвига


In [131]:
# оригинальный код вот тут - https://norvig.com/spell-correct.html
# я только адаптировал его под русский язык

def correction(word):
    closest = candidates(word)
    closest = {
        word: {
            "P" : P(word),
        }
        for word
        in closest
    }
    "Находим наиболее вероятное похожее слово"
    return max(closest, key=lambda c: closest[c]["P"])

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in vocab)


def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [132]:
correction("cсолнце")

'солнце'

In [133]:
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [134]:
from typing import Callable
from tqdm import tqdm

class Tester:

    bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
    true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

    @classmethod
    def get_metrics(cls, correction_func: Callable) -> str:
        correct = 0
        total = 0

        total_mistaken = 0
        mistaken_fixed = 0

        total_correct = 0
        correct_broken = 0

        cashed = {}
        for i in tqdm(range(len(cls.true))):
            word_pairs = align_words(cls.true[i], cls.bad[i])
            for pair in word_pairs:
                # чтобы два раза не исправлять одно и тоже слово - закешируем его
                # перед тем как считать исправление проверим нет ли его в кеше
                
                predicted = cashed.get(pair[1], correction_func(pair[1]))
                cashed[pair[1]] = predicted
                
                
                if predicted == pair[0]:
                    correct += 1
                total += 1
                
                if pair[0] == pair[1]:
                    total_correct += 1
                    if pair[0] !=  predicted:
                        correct_broken += 1
                else:
                    total_mistaken += 1
                    if pair[0] == predicted:
                        mistaken_fixed += 1

        return (
            correct/total,
            mistaken_fixed/total_mistaken,
            correct_broken/total_correct,
        )

In [135]:
Tester.get_metrics(
    lambda word: get_closest_hybrid_match(word, X, vec, topn=4)
)

100%|██████████| 915/915 [07:55<00:00,  1.92it/s]


(0.856328164082041, 0.4937888198757764, 0.09004249454461927)

In [136]:
Tester.get_metrics(correction)

  0%|          | 0/915 [00:00<?, ?it/s]

100%|██████████| 915/915 [01:22<00:00, 11.10it/s]


(0.870935467733867, 0.5124223602484472, 0.07603077983231882)

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [ ]:
class Symspell:
    
    @staticmethod
    def get_deletes(word: str) -> set[str]:
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = {L + R[1:]            for L, R in splits if R}
        return deletes

    del_dict: dict[str, str] = { }
    for word in sorted(tuple(vocab), key=lambda c: vocab[c]):  ##  так в словаре окажутся наиболее частотные варианты
        for delete in get_deletes(word):
            del_dict[delete] = word
        del_dict[word] = word
    del_set = set(del_dict)

    @classmethod
    def correct(cls, word:str) -> str:

        if word in vocab:
            return word

        deletes     : set[str]   = cls.get_deletes(word)
        intersection: tuple[str] = tuple(deletes & cls.del_set)

        inter_len = len(intersection)
        if inter_len == 0:
            return word
        elif inter_len == 1:
            return cls.del_dict[intersection[0]]
        else:
            return max([cls.del_dict[w] for w in intersection], key=P)


In [138]:
Symspell.correct("cсолнце")

'солнце'

In [139]:
Tester.get_metrics(
    Symspell.correct
)

100%|██████████| 915/915 [00:00<00:00, 35244.63it/s]


(0.8666333166583292, 0.30434782608695654, 0.05018950269897783)

In [140]:
class Symspell:
    
    @staticmethod
    def get_deletes(word: str) -> set[str]:
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = {L + R[1:]            for L, R in splits if R}
        return deletes

    del_dict: dict[str, str] = { }
    for word in vocab:  ##  так в словаре окажутся наиболее частотные варианты
        for delete in get_deletes(word):
            del_dict[delete] = word
        del_dict[word] = word
    del_set = set(del_dict)

    @classmethod
    def correct(cls, word:str) -> str:

        if word in vocab:
            return word

        deletes     : set[str]   = cls.get_deletes(word)
        intersection: tuple[str] = tuple(deletes & cls.del_set)

        inter_len = len(intersection)
        if inter_len == 0:
            return word
        elif inter_len == 1:
            return cls.del_dict[intersection[0]]
        else:
            return max([cls.del_dict[w] for w in intersection], key=P)

In [ ]:
Tester.get_metrics(
    Symspell.correct
)